# Titanic Survival Prediction

## Context

On April 15, 1912, the Titanic collided into an iceberg and sank. Out of 2224 passengers and crew on the ship, 1502 of them unfortunately died.

## Objective

The Titanic included many different types of people, which may have played a factor in whether or not each person survived. The object of this project is to create a model that can predict if someone survived the crash based on these various factors.

## Data Overview

Let's start by importing the data and various packages that will help with our analysis and modeling.

Given that we want to predict whether or not someone survived the Titanic crash, we will want to use classification modeling. We will explore both a logistic regression model and a decision tree model.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # data visualization

# Removes the limit from the number of displayed columns and rows.
# This is so I can see the entire dataframe when I print it
pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', None)
pd.set_option("display.max_rows", 200)

# To filter the warnings
import warnings

warnings.filterwarnings("ignore")

# To build linear model for statistical analysis and prediction
import scipy.stats as stats
import statsmodels.stats.api as sms
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant

# Libraries to build decision tree classifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import tree

# To tune different models
from sklearn.model_selection import GridSearchCV

# To get diferent metric scores
from sklearn import metrics
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    recall_score,
    precision_score,
    confusion_matrix,
    plot_confusion_matrix,
    make_scorer,
    roc_curve,
    roc_auc_score,
    precision_recall_curve
)



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory




import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Import the train data and the test data

In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

Let's see some quick observations we can make in the data.

In [4]:
train_data.info()

In [5]:
train_data.shape

In [6]:
train_data.describe(include="all").T

In [7]:
# checking for null values
train_data.isnull().sum()

In [8]:
# checking for duplicate values
train_data.duplicated().sum()

##### Observations:
- We should remove the PassengerId and Name columns from the data before entering into the model. These variables are unique for every person, so it doesn't make sense to make predictions based off those variables.
- There are no duplicate rows.
- Out of 891 rows in the training data, there are 687 null values for the Cabin column. It probably doesn't make sense to make predictions off of a variable with not much information, so we should just remove that column.
- There are 177 null values for the Age column. I will likely populate the null values with the median value of the Age column.
- There are 2 null values for the Embarked column. I may either keep that as is or populate with the mode value of the Embarked column.
- The Pclass column is currently an int column. This is potentially ok because there is a quantifiable difference between the ticket classes (e.g. 1st is the highest class, 3rd is the lowest class). However, it may be valuable to convert this column to a category column and then one-hot encode it.
- The Fare column will likely have multicollinearity with the Pclass column, because the passenger fare is likely higher for 1st class compared to other classes.
- We should delete the Ticket column. The ticket number value doesn't provide quantifiable difference between rows, and there are 681 unique Ticket column values. Any row that shares ticket numbers are likely in the same party and will be considered from the Sibsp and Parch columns.
- Overall, there is definitely some data cleaning to do. We will do Exploratory Data Analysis to confirm if other data cleaning should occur.

## Exploratory Data Analysis & Data Cleaning

In [9]:
# Remove the PassengerId, Name, Ticket, and Cabin columns
cols = ["PassengerId","Name","Ticket", "Cabin"]
for col in cols:
    train_data = train_data.drop([col], axis=1)

In [10]:
train_data.head()

#### Creating a couple functions to help with visualization:

In [11]:
# function to plot a boxplot and a histogram along the same scale.


def histogram_boxplot(data, feature, figsize=(12, 7), kde=False, bins=None):
    """
    Boxplot and histogram combined

    data: dataframe
    feature: dataframe column
    figsize: size of figure (default (12,7))
    kde: whether to the show density curve (default False)
    bins: number of bins for histogram (default None)
    """
    f2, (ax_box2, ax_hist2) = plt.subplots(
        nrows=2,  # Number of rows of the subplot grid= 2
        sharex=True,  # x-axis will be shared among all subplots
        gridspec_kw={"height_ratios": (0.25, 0.75)},
        figsize=figsize,
    )  # creating the 2 subplots
    sns.boxplot(
        data=data, x=feature, ax=ax_box2, showmeans=True, color="violet"
    )  # boxplot will be created and a star will indicate the mean value of the column
    sns.histplot(
        data=data, x=feature, kde=kde, ax=ax_hist2, bins=bins, palette="winter"
    ) if bins else sns.histplot(
        data=data, x=feature, kde=kde, ax=ax_hist2
    )  # For histogram
    ax_hist2.axvline(
        data[feature].mean(), color="green", linestyle="--"
    )  # Add mean to the histogram
    ax_hist2.axvline(
        data[feature].median(), color="black", linestyle="-"
    )  # Add median to the histogram

In [12]:
# function to create labeled barplots


def labeled_barplot(data, feature, perc=False, n=None):
    """
    Barplot with percentage at the top

    data: dataframe
    feature: dataframe column
    perc: whether to display percentages instead of count (default is False)
    n: displays the top n category levels (default is None, i.e., display all levels)
    """

    total = len(data[feature])  # length of the column
    count = data[feature].nunique()
    if n is None:
        plt.figure(figsize=(count + 2, 6))
    else:
        plt.figure(figsize=(n + 2, 6))

    plt.xticks(rotation=90, fontsize=15)
    ax = sns.countplot(
        data=data,
        x=feature,
        palette="Paired",
        order=data[feature].value_counts().index[:n],
    )

    for p in ax.patches:
        if perc == True:
            label = "{:.1f}%".format(
                100 * p.get_height() / total
            )  # percentage of each class of the category
        else:
            label = p.get_height()  # count of each level of the category

        x = p.get_x() + p.get_width() / 2  # width of the plot
        y = p.get_height()  # height of the plot

        ax.annotate(
            label,
            (x, y),
            ha="center",
            va="center",
            size=12,
            xytext=(0, 5),
            textcoords="offset points",
        )  # annotate the percentage

    plt.show()  # show the plot

In [13]:
def stacked_barplot(data, predictor, target):
    """
    Print the category counts and plot a stacked bar chart

    data: dataframe
    predictor: independent variable
    target: target variable
    """
    count = data[predictor].nunique()
    sorter = data[target].value_counts().index[-1]
    tab1 = pd.crosstab(data[predictor], data[target], margins=True).sort_values(
        by=sorter, ascending=False
    )
    print(tab1)
    print("-" * 120)
    tab = pd.crosstab(data[predictor], data[target], normalize="index").sort_values(
        by=sorter, ascending=False
    )
    tab.plot(kind="bar", stacked=True, figsize=(count + 5, 5))
    plt.legend(
        loc="lower left", frameon=False,
    )
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.show()

In [14]:
### function to plot distributions wrt target


def distribution_plot_wrt_target(data, predictor, target):

    fig, axs = plt.subplots(2, 2, figsize=(12, 10))

    target_uniq = data[target].unique()

    axs[0, 0].set_title("Distribution of target for target=" + str(target_uniq[0]))
    sns.histplot(
        data=data[data[target] == target_uniq[0]],
        x=predictor,
        kde=True,
        ax=axs[0, 0],
        color="teal",
        stat="density",
    )

    axs[0, 1].set_title("Distribution of target for target=" + str(target_uniq[1]))
    sns.histplot(
        data=data[data[target] == target_uniq[1]],
        x=predictor,
        kde=True,
        ax=axs[0, 1],
        color="orange",
        stat="density",
    )

    axs[1, 0].set_title("Boxplot w.r.t target")
    sns.boxplot(data=data, x=target, y=predictor, ax=axs[1, 0], palette="gist_rainbow")

    axs[1, 1].set_title("Boxplot (without outliers) w.r.t target")
    sns.boxplot(
        data=data,
        x=target,
        y=predictor,
        ax=axs[1, 1],
        showfliers=False,
        palette="gist_rainbow",
    )

    plt.tight_layout()
    plt.show()

##### What is the age distribution across the people on the Titanic?

In [15]:
histogram_boxplot(train_data, "Age")

There is a spike in people with age close to zero, which means there are a good number of infants on the ship.

Given that the age distribution is generally a normal distribution with most people being in the 20s and 30s, and the mean being close to the median, I'm comfortable with replacing all the null age values with 30.

In [16]:
train_data['Age'].replace(np.nan,30,inplace=True)

In [17]:
train_data.info()

##### Does Port of Embarkation have an impact of survival status of a person?

In [18]:
stacked_barplot(train_data, "Embarked", "Survived")

In [19]:
labeled_barplot(train_data, "Embarked", perc=True)

It appears like people with Cherbourg as the Port of Embarkation were more likely to survive compared to Queenstown and Southampton. 

Given that we only have 2 null values in the train data and Southampton encompasses almost three-fourths of the train data, I'm ok with replacing the null values with Southampton.

In [20]:
train_data['Embarked'].replace(np.nan,"S",inplace=True)

##### Does sex have an impact on the survival rate of a person?

In [21]:
stacked_barplot(train_data, "Sex", "Survived")

In [22]:
labeled_barplot(train_data, "Sex", perc=True)

The distribution of sex on the Titanic was about two-thirds male and one-third female. However, the survival percentage was a lot higher for females than males. 
This is consistent with the movie, where women and children were prioritized escape!

##### How many attendees on the ship had others in their party?

In [23]:
histogram_boxplot(train_data, "SibSp")

In [24]:
histogram_boxplot(train_data, "Parch")

In [25]:
distribution_plot_wrt_target(train_data,"SibSp","Survived")

In [26]:
distribution_plot_wrt_target(train_data,"Parch","Survived")

Most attendees on the ship were alone. This is likely skewed to 0 because the data set includes crew members as well.

There appears to be no correlation between number of siblings/spouses and survival rate.
However, people with parents/children seem to be more likely to survive.



Let's briefly assume that a crew member could be identified by if the Fare was 0. Would this change the distribution and survival status?

In [27]:
histogram_boxplot(train_data[train_data["Fare"]!=0], "SibSp")

In [28]:
histogram_boxplot(train_data[train_data["Fare"]!=0], "Parch")

In [29]:
distribution_plot_wrt_target(train_data[train_data["Fare"]!=0],"SibSp","Survived")

In [30]:
distribution_plot_wrt_target(train_data[train_data["Fare"]!=0],"Parch","Survived")

After removing rows where Fare was zero (because they were assumed as crew members), there didn't appear to be a major change in the distribution and survival status.

Beyond Fare = zero, is there an correlation for Fare with survival rate?



In [31]:
histogram_boxplot(train_data, "Fare")

In [32]:
distribution_plot_wrt_target(train_data,"Fare","Survived")

Generally, those that paid a higher fare were more likely to survive.

##### Is there a correlation between Pclass and Fare?

In [33]:
distribution_plot_wrt_target(train_data,"Fare","Pclass")

Yes, people in a ticket class of 1 generally paid a higher fare. People in a ticket class of 3 generally paid the lowest for fare. These may be multicollinear variables.

Now that we got a good understanding and visualization of the data and we cleaned appropriate places, let's prepare the data for modeling!

# Data Preparation

In [34]:
X_train = train_data.drop(["Survived"],axis=1)
y_train = train_data["Survived"]

# Logistic Regression Modeling

Logistic Regression modeling will require a little more data preparation. We need to add a constant and one-hot encode categorical variables.

In [35]:
# adding a contstant to X variable
X_log1 = add_constant(X_train)

# creating dummies
X_log1 = pd.get_dummies(X_log1, drop_first=True)

In [36]:
# fitting the model on training set
logit1 = sm.Logit(y_train, X_log1.astype(float))
lg1 = logit1.fit()

In [37]:
# let's print the logistic regression summary
print(lg1.summary())

##### Observations:
- Embarked_Q, Parch, and Fare have high p-value, so there's likely some multicollinearity at play and it's likely worth removing some of them.

Before doing further enhancements to this model, let's check the model performance with a confusion matrix.

Generally, I care about decreasing false positives and false negatives equally. So I will primarily focus on using the F1 score to track model performance too.

We will also set a default threshold of 0.5.

In [38]:
def conf_matrix_and_scores(X_train, y_train, lg, threshold=0.5):
    """
    Make a confusion matrix and calculate metrics
    X_train: The X train data set
    y_train: The y train data set
    lg: the logistic regression model
    threshold: threshold to use for the model (default 0.5)
    """
    pred_train = lg.predict(X_train) > threshold
    pred_train = np.round(pred_train)
    cm = confusion_matrix(y_train, pred_train)
    plt.figure(figsize=(7, 5))
    sns.heatmap(cm, annot=True, fmt="g")
    plt.xlabel("Predicted Values")
    plt.ylabel("Actual Values")
    plt.show()
    
    print("Accuracy on training set : ", accuracy_score(y_train, pred_train))
    print("Recall on training set: ", recall_score(y_train, pred_train))
    print("Precision on training set: ", precision_score(y_train, pred_train))
    print("F1 score on training set: ", f1_score(y_train, pred_train))

In [39]:
conf_matrix_and_scores(X_log1, y_train, lg1)

An F1 score of 73% is good. But there are a good number of false positives and false negatives. Let's handle that better through some data cleaning. Starting with multicollinearity calculations.

In [40]:
# let's check the VIF of the predictors
vif_series = pd.Series(
    [variance_inflation_factor(X_log1.values, i) for i in range(X_log1.shape[1])],
    index=X_log1.columns,
    dtype=float,
)
print("VIF values: \n\n{}\n".format(vif_series))

Interestingly, there is no strong multicollinearity in the data. I expected at least Pclass or Fare to be related.

I will still remove the Embarked_Q column and make a new model.

In [41]:
X_log2 = X_log1.drop(["Embarked_Q"],axis=1)
logit2 = sm.Logit(y_train, X_log2.astype(float))
lg2 = logit2.fit()
print(lg2.summary())

In [42]:
conf_matrix_and_scores(X_log2, y_train, lg2)

As expected, p-values are still intact.

Let's go through our metrics calculations of this model.

No major change in metrics.

Parch has the highest p-value, so I will remove that now and model again.

In [43]:
X_log3 = X_log2.drop(["Parch"],axis=1)
logit = sm.Logit(y_train, X_log3.astype(float))
lg3 = logit.fit()
print(lg3.summary())

In [44]:
conf_matrix_and_scores(X_log3, y_train, lg3)

No major change in metrics.

Fare sill has a high p-value, so I will remove that now and model again.

In [45]:
X_log4 = X_log3.drop(["Fare"],axis=1)
logit = sm.Logit(y_train, X_log4.astype(float))
lg4 = logit.fit()
print(lg4.summary())

In [46]:
conf_matrix_and_scores(X_log4, y_train, lg4)

We will keep lg4 as our logistic regression model. Let's intrepret what this model tells us.

* The coefficients of the logistic regression model are in terms of log(odd), to find the odds we have to take the exponential of the coefficients. 
* Therefore, **odds =  exp(b)**
* The percentage change in odds is given as **odds = (exp(b) - 1) * 100**

In [47]:
# converting coefficients to odds
odds = np.exp(lg4.params)

# adding the odds to a dataframe
pd.DataFrame(odds, X_log4.columns, columns=["odds"]).T

In [48]:
# finding the percentage change
perc_change_odds = (np.exp(lg4.params) - 1) * 100

# adding the change_odds% to a dataframe
pd.DataFrame(perc_change_odds, X_log4.columns, columns=["change_odds%"]).T

##### Coefficient Interpretations:

- "Pclass": Holding all other features constant, a unit change in Pclass will decrease the odds of survival by 68.80%
- "Age": Holding all other features constant, a unit change in Age will decrease the odds of survival by 3.90%
- "SibSp": Holding all other features constant, a unit change in the number of siblings our spouses in your part will decrease the odds of survival by 28.46%
- "Sex": Holding all other features constant, being a male will decrease the odds of survival by 93.28%.
- "Embarked": Holding all other features constant, embarking from Southampton will decrease the odds of survival by 35.87%.

Let's see if we can enhance the model further with another threshold value.

In [49]:
logit_roc_auc_train = roc_auc_score(y_train, lg4.predict(X_log4))
fpr, tpr, thresholds = roc_curve(y_train, lg4.predict(X_log4))
plt.figure(figsize=(7, 5))
plt.plot(fpr, tpr, label="Logistic Regression (area = %0.2f)" % logit_roc_auc_train)
plt.plot([0, 1], [0, 1], "r--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic")
plt.legend(loc="lower right")
plt.show()

In [50]:
y_scores = lg4.predict(X_log4)
prec, rec, tre = precision_recall_curve(y_train, y_scores)


def plot_prec_recall_vs_tresh(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="precision")
    plt.plot(thresholds, recalls[:-1], "g--", label="recall")
    plt.xlabel("Threshold")
    plt.legend(loc="upper left")
    plt.ylim([0, 1])


plt.figure(figsize=(10, 7))
plot_prec_recall_vs_tresh(prec, rec, tre)
plt.show()

0.5 does seem to be the ideal value for the threshold. We'll keep as is.

Now let's explore a decision tree model!

# Decision Tree Modeling

In [51]:
# creating dummies
X_dt = pd.get_dummies(X_train, drop_first=True)

dt1 = DecisionTreeClassifier(random_state=1)
dt1.fit(X_dt, y_train)

We can actually use the same metric calculation function that was used in logistic regression.

In [52]:
conf_matrix_and_scores(X_dt, y_train, dt1)

This decision tree model looks great, but a problem with this type of modeling is that it is prone to overfitting.

Let's make another decision tree model with a balanced class weight.

In [53]:
dt2 = DecisionTreeClassifier(random_state=1, class_weight="balanced") 
dt2.fit(X_dt, y_train)

In [54]:
conf_matrix_and_scores(X_dt, y_train, dt2)

Let's visualize what this model looks like.

In [55]:
plt.figure(figsize=(20, 30))

out = tree.plot_tree(
    dt2,
    feature_names=list(X_dt.columns),
    filled=True,
    fontsize=9,
    node_ids=True,
    class_names=True,
)
for o in out:
    arrow = o.arrow_patch
    if arrow is not None:
        arrow.set_edgecolor("black")
        arrow.set_linewidth(1)
plt.show()

There is clearly a lot of nodes here, clearly overcomplex and overfitting so it should be pruned.

First, let's get a review of the variable importances.

In [56]:
importances = dt2.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(12, 12))
plt.title("Feature Importances")
plt.barh(range(len(indices)), importances[indices], color="violet", align="center")
plt.yticks(range(len(indices)), [list(X_dt.columns)[i] for i in indices])
plt.xlabel("Relative Importance")
plt.show()

Sex, Fare, and Age were the highest contributors to the decision tree model. This is pretty different from the logistic regression model we solidified - let's see if this changes as we prune the decision tree model.

We'll start with pre-pruning. We will circle through the list of various hyperparameters until we get the one with the best F1 score.

In [ ]:
# Choose the type of classifier.
estimator = DecisionTreeClassifier(random_state=1)

# Grid of parameters to choose from

parameters = {
    "criterion": ["entropy", "gini"],
    "max_depth": np.arange(2, 11, 2),   
    "max_leaf_nodes": [50, 75, 150, 250],
    "min_samples_split": [10, 30, 50, 70],
    "min_impurity_decrease": [0.000001, 0.00001, 0.0001],
}

# Type of scoring used to compare parameter combinations
acc_scorer = make_scorer(f1_score)

# Run the grid search
grid_obj = GridSearchCV(estimator, parameters, scoring=acc_scorer, cv=5)
grid_obj = grid_obj.fit(X_dt, y_train)

# Set the clf to the best combination of parameters
estimator = grid_obj.best_estimator_

# Fit the best algorithm to the data.
estimator.fit(X_dt, y_train)

Our iterative process showed us that the following would be ideal for a decision tree model:
* Criterion of entropy
* Max depth of 10
* Max leaf node number as 50
* Min samples needed to split as 30
* Min impurity decrease as 1e-6

In [ ]:
conf_matrix_and_scores(X_dt, y_train, estimator)

In [ ]:
plt.figure(figsize=(20, 30))

out = tree.plot_tree(
    estimator,
    feature_names=list(X_dt.columns),
    filled=True,
    fontsize=9,
    node_ids=True,
    class_names=True,
)
for o in out:
    arrow = o.arrow_patch
    if arrow is not None:
        arrow.set_edgecolor("black")
        arrow.set_linewidth(1)
plt.show()

In [ ]:
importances = estimator.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(12,12))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='violet', align='center')
plt.yticks(range(len(indices)), [list(X_dt.columns)[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

The pre-pruned model is a lot more simplified. The new F1 score is 81%, which is less than before but also is less prone to overfitting.
The pre-pruned model also prioritizes sex the most of all the features.

##### Cost Complexity Pruning:

Let's visualize the effect of the cost complexity parameter with the decision tree modeling. We will recursively find the node with the weakest link by returning the effective alphas and corresponding total leaf impurities at each step. 

In [ ]:
clf = DecisionTreeClassifier(random_state=1)
path = clf.cost_complexity_pruning_path(X_dt, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

In [ ]:
path = pd.DataFrame(path)
path

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(ccp_alphas[:-1], impurities[:-1], marker="o", drawstyle="steps-post")
ax.set_xlabel("effective alpha")
ax.set_ylabel("total impurity of leaves")
ax.set_title("Total Impurity vs effective alpha for training set")
plt.show()

Now we will train a decision tree to use the effective alphas. We'll find the alpha that provides the best F1 score.

In [ ]:
clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=1, ccp_alpha=ccp_alpha)
    clf.fit(X_dt, y_train)
    clfs.append(clf)
print("Number of nodes in the last tree is: {} with ccp_alpha: {}".format(
      clfs[-1].tree_.node_count, ccp_alphas[-1]))

We should obviously remove the last value, because a decision tree with only 1 node doesn't provide any information.

In [ ]:
clfs = clfs[:-1]
ccp_alphas = ccp_alphas[:-1]

node_counts = [clf.tree_.node_count for clf in clfs]
depth = [clf.tree_.max_depth for clf in clfs]
fig, ax = plt.subplots(2, 1,figsize=(10,7))
ax[0].plot(ccp_alphas, node_counts, marker='o', drawstyle="steps-post")
ax[0].set_xlabel("alpha")
ax[0].set_ylabel("number of nodes")
ax[0].set_title("Number of nodes vs alpha")
ax[1].plot(ccp_alphas, depth, marker='o', drawstyle="steps-post")
ax[1].set_xlabel("alpha")
ax[1].set_ylabel("depth of tree")
ax[1].set_title("Depth vs alpha")
fig.tight_layout()

In [ ]:
f1_train = []
for clf in clfs:
    pred_train=clf.predict(X_dt)
    values_train=metrics.f1_score(y_train,pred_train)
    f1_train.append(values_train)
fig, ax = plt.subplots(figsize=(10,5))
ax.set_xlabel("alpha")
ax.set_ylabel("F1 Score")
ax.set_title("F1 Score vs alpha for training set")
ax.plot(ccp_alphas, f1_train, marker='o', label="train",
        drawstyle="steps-post")
ax.legend()
plt.show()

From the above graphs, it looks like an alpha of around 0.004 would be best. Higher than that will cause exponential changes to number of nodes and depth. Lower than that risks basing the model on noise and overfitting.
To get the exact number, I will grab from the 76th row of the path dataframe.

In [ ]:
ideal_model=clfs[76]
print(ideal_model)

In [ ]:
conf_matrix_and_scores(X_dt, y_train, ideal_model)

In [ ]:
importances = ideal_model.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(12,12))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='violet', align='center')
plt.yticks(range(len(indices)), [list(X_dt.columns)[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
plt.figure(figsize=(20, 30))

out = tree.plot_tree(
    ideal_model,
    feature_names=list(X_dt.columns),
    filled=True,
    fontsize=9,
    node_ids=True,
    class_names=True,
)
for o in out:
    arrow = o.arrow_patch
    if arrow is not None:
        arrow.set_edgecolor("black")
        arrow.set_linewidth(1)
plt.show()

# Model Performance Comparison and Conclusions

- We'll use the post-pruned Decision Tree model for our test data set. This is chosen because it is a simple to understand model (depth of only 3 and only 7 leaf nodes) and yet it provides a solid F1 score of 76%.
- According to this model, the sex of the person is the strongest identifier for survival status, followed by Pclass. This makes sense because women were generally prioritized escape, and richer people probably had more power to influence prioritizing themselves over others.
- To make a stronger model, the following can be done for the future:
** Information about if someone was a guest or on the crew. This likely would be a strong factor, as crew members would have known the requirements to get to safety. This data can likely be gathered from other sources.
** We removed the Cabin column due to having many null values. However, this could be a strong factor too because those that lives lower in the ship would have had to go through more effort to not drown. We can try to find reliable information for this data in other sources, or model with only the rows that have this information.

# Running Model on Test Data

First, we need to clean the test data set in the same way that we did it for the train data set before modeling.
We will copy into a new data frame to avoid breaking the test data set.

In [ ]:
test_data.head()

In [ ]:
test_data.info()

##### Things to clean:
- Just like the train set, remove the columns for PassengerId, Name, Ticket, Cabin
- Replace null values for Age with the value of 30.
- No need to replace null values in the Embarked column because that already has all values populated.
- There is one row with a null Fare value. We will replace that with a 0 because that is the mode of the Fare column.
- One-hot encode the categorical variables.

In [ ]:
# Remove the PassengerId, Name, Ticket, and Cabin columns
cols = ["PassengerId","Name","Ticket", "Cabin"]
X_testdt=test_data
for col in cols:
    X_testdt = X_testdt.drop([col], axis=1)
    
# Handle null values
X_testdt['Age'].replace(np.nan,30,inplace=True)
X_testdt['Fare'].replace(np.nan,0,inplace=True)
    
# One-hot encode variables
X_testdt = pd.get_dummies(X_testdt, drop_first=True)

In [ ]:
# Confirm we have a test data set in the correct format and no null values
X_testdt.info()

In [ ]:
# Quick glance at part of the test data set.
X_testdt.head()

Now we can make our submission file!

In [ ]:
predictions = ideal_model.predict(X_testdt)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")